In [24]:
# Might need to install the packages below using 
# "pip install urllib" 
#  and 
# "pip install arxiv"
#  and 
# "pip install unidecode"

import urllib.request
import arxiv
import unidecode


def DOI_2_bib(doi):    
    doi_req = urllib.request.Request('http://dx.doi.org/' + doi)
    doi_req.add_header('Accept', 'application/x-bibtex') 
    with urllib.request.urlopen(doi_req) as f:
        print(f.read().decode())
        # return f.read().decode()
    
    
def arXib_bib_format(paper):
    author_list = ' and '.join([str(author) for author in paper.authors])
    citation_key = str(paper.authors[0]).split(' ')[-1] + '_' + str(paper.published.year)
    citation_key = unidecode.unidecode(citation_key)
    return f'''@article{{{citation_key},
         title = {{{paper.title}}},
         author = {{{author_list.strip()} }},
         year = {{{paper.published.year}}},
         eprint = {{{paper.get_short_id()}}}, 
         archivePrefix = {{arXiv}},      
         primaryClass ={{{paper.categories[0]}}}
}}
    '''

def arXiv_2_bib(ID):
    search = arxiv.Search(id_list=[ID])
    paper = next(search.results())
    doi = paper.doi
    try: 
        return DOI_2_bib(doi)
    except: 
        return print(arXib_bib_format(paper))    

In [35]:
DOI_2_bib('10.1038/s41567-020-1018-2')

@article{Seif_2020,
	doi = {10.1038/s41567-020-1018-2},
	url = {https://doi.org/10.1038%2Fs41567-020-1018-2},
	year = 2020,
	month = {sep},
	publisher = {Springer Science and Business Media {LLC}},
	volume = {17},
	number = {1},
	pages = {105--113},
	author = {Alireza Seif and Mohammad Hafezi and Christopher Jarzynski},
	title = {Machine learning the thermodynamic arrow of time},
	journal = {Nature Physics}
}


In [36]:
DOI_2_bib('10.1103/PhysRevLett.125.080601')

@article{Rossi_2020,
	doi = {10.1103/physrevlett.125.080601},
	url = {https://doi.org/10.1103%2Fphysrevlett.125.080601},
	year = 2020,
	month = {aug},
	publisher = {American Physical Society ({APS})},
	volume = {125},
	number = {8},
	author = {Massimiliano Rossi and Luca Mancino and Gabriel T. Landi and Mauro Paternostro and Albert Schliesser and Alessio Belenchia},
	title = {Experimental Assessment of Entropy Production in a Continuously Measured Mechanical Resonator},
	journal = {Physical Review Letters}
}


In [34]:
arXiv_2_bib('1610.02244')

@article{Al_Assam_2017,
	doi = {10.1088/1742-5468/aa7df3},
	url = {https://doi.org/10.1088%2F1742-5468%2Faa7df3},
	year = 2017,
	month = {sep},
	publisher = {{IOP} Publishing},
	volume = {2017},
	number = {9},
	pages = {093102},
	author = {S Al-Assam and S R Clark and D Jaksch},
	title = {The tensor network theory library},
	journal = {Journal of Statistical Mechanics: Theory and Experiment}
}


In [4]:
# If the paper is not published, it will output the arXiv information
arXiv_2_bib('cond-mat/0007360')

@article{Kurchan_2000,
         title = {A Quantum Fluctuation Theorem},
         author = {Jorge Kurchan },
         year = {2000},
         eprint = {cond-mat/0007360v2}, 
         archivePrefix = {arXiv},      
         primaryClass ={cond-mat.stat-mech}
}
    
